## SUPPLY CHAIN NETWORK DESIGN TO SUPPORT BIOFUEL PRODUCTION
<p> A company has decided to produce bioethanol in the state of Texas. The company needs to design a supply chain consisting of suppliers, hubs and biorefineries for the conversion of raw material (i.e., biomass) into biofuel. </p>
<p> The potential locations to open hubs correspond to train stations because the transportation mode utilized to move the raw material from the hubs to the biorefineries is train, while truck is the transportation mode utilized to move the biomass from the counties to the hubs.</p>
<p> This project is to minimize the investment and transportation costs by finding the optimal number of hubs and biorefineries that the company needs to install as well as the flows between suppliers-hubs and hubs-biorefineries.</p>


<p> PuLP is the Python library installed, imported and used in this project for linear optimization </p>

In [1]:
# Import the necessary libraries
import numpy as np
import math
import pandas as pd
import pulp
from pulp import*

## Data Importation and Formatting

In [2]:
suppliers = pd.read_csv('TX_suppliers.csv')
suppliers['supply'] = round(suppliers.supply, 2)
suppliers = suppliers.drop(['index'], axis =1)
supplies = suppliers.supply
total_supply = sum(suppliers.supply)

hubs = pd.read_csv('TX_hubs.csv')
Hub_Capacity = hubs.capacity
Hub_Invest = hubs.invest.iloc[0]
Hubs = hubs['hub']

road = pd.read_csv('TX_roads.csv')
road['cost'] = round(road.cost, 2)
road = road.drop(['index'], axis =1)

plants = pd.read_csv('TX_plants.csv')
Yield = plants['yield'].iloc[0]
plants['plt_capacity'] = plants.capacity/plants['yield']
plants = plants.drop(['index'], axis =1)
Plt_Invest = plants.invest.iloc[0]

rail = pd.read_csv('TX_railroads.csv')
rail['cost'] = round(rail.cost, 2)
rail['plt_invest'] = len(rail)*[plants.invest.iloc[0]]
rail = rail.drop(['index'], axis =1)
loading = rail.loading

network = pd.read_csv('TX_network.csv')
Demand = network.demand.iloc[0]
demand = Demand/Yield

unmet_Demand = demand - total_supply

## Introducing 3rd Party Supplier
<p> Since the sum of the county supplies can not meet the demand, a 3rd party supplier is introduced with an estimated unit cost </p>

In [3]:
suppliers.loc[254] = ['3_party', unmet_Demand]
suppliers['supply'] = round(suppliers.supply, 2)

In [4]:
# 3rd Party Estimate Cost
Third_cost = 29.96
Third_cost = round(Third_cost)

In [5]:
# Adding 3rd Party Supplier to road table
n = len(Hubs)
County = ['3_party'] * n
Distance = [0] * n
Cost = [Third_cost] * n
Third = {'county': County, 'distance':Distance, 'cost':Cost}
Third = pd.DataFrame(Third)
Third['hubs'] = Hubs

In [6]:
road = road.append(Third, ignore_index = True) 

In [7]:
# Setting the indices
hubs = hubs.set_index(['hub'])
suppliers = suppliers.set_index(['county'])
plants = plants.set_index(['plant'])
road = road.set_index(['county', 'hubs'])
rail = rail.set_index(['hubs','plant'])

In [8]:
suppliers.head()

,supply
county,
48001,13131.97
48003,1177.35
48005,3854.62
48007,308.18
48009,19802.14


In [9]:
road.head()

distance    cost
county hubs                   
48001  131   1218.9736  197.28
       199    299.0238   53.03
       229    899.4465  147.18
       512    300.1810   53.21
       560    299.0745   53.04

In [10]:
hubs.head()

,latitude,longitude,invest,capacity
hub,,,,
131,32.00055,-106.60614,3476219,300000
199,33.41906,-94.04270,3476219,300000
229,36.50022,-102.24470,3476219,300000
512,33.41778,-94.04277,3476219,300000
560,33.41962,-94.04271,3476219,300000


In [11]:
rail.head()

distance   cost  loading  capacity  plt_invest
hubs plant                                                   
131  541    1846.182620  31.91  3066792    338000   130956797
     542    1698.400668  29.54  3066792    338000   130956797
     543    1970.225660  33.90  3066792    338000   130956797
     544    1693.506798  29.46  3066792    338000   130956797
     545    1698.400668  29.54  3066792    338000   130956797

In [12]:
plants.head()

,latitude,longitude,tech,invest,capacity,yield,plt_capacity
plant,,,,,,,
541,30.220353,-97.740436,1,130956797,152063705,232,655447.00431
542,29.676724,-98.635234,1,130956797,152063705,232,655447.00431
543,33.760315,-96.559867,1,130956797,152063705,232,655447.00431
544,29.531920,-98.286901,1,130956797,152063705,232,655447.00431
545,29.688568,-98.562424,1,130956797,152063705,232,655447.00431


## Data Exploration and Optimization

In [13]:
# Decision Variables Defintion
road_supply = pulp.LpVariable.dicts('road_supply', [(i, j) for i in suppliers.index
                                                    for j in hubs.index], lowBound=0, cat='Continuous')

hub_status = pulp.LpVariable.dicts("hub_status", [j for j in hubs.index], cat='Binary')

rail_supply = pulp.LpVariable.dicts("rail_supply",[(j, k) for j in hubs.index
                                                   for k in plants.index], lowBound=0, cat='Continuous')

plt_status = pulp.LpVariable.dicts("plt_status",[k for k in plants.index], cat='Binary')

## Model

In [14]:
# Model Initialization
model = pulp.LpProblem("cost minimising supply network", pulp.LpMinimize)

In [15]:
# Objective Function
model += pulp.lpSum(
    [[road_supply[i, j] * road.loc[(i, j), 'cost']]  + 
     [hub_status[j] * hubs.loc[j, 'invest']] for i in suppliers.index for j in hubs.index] + [[rail_supply[j, k] * rail.loc[(j, k), 'cost']] + 
     [plt_status[k] * plants.loc[k, 'invest']] for j in hubs.index for k in plants.index]) 

In [16]:
# Demand Constraints
model += pulp.lpSum([road_supply[i, j] for i in suppliers.index for j in hubs.index]) == demand
model += pulp.lpSum([rail_supply[j, k] for j in hubs.index for k in plants.index]) == demand

In [17]:
# Hub Capacity Constraint
for j in hubs.index:
    model += pulp.lpSum([road_supply[i, j] for i in suppliers.index]) <= hubs.loc[j, 'capacity'] * hub_status[j]

In [18]:
# Plant Capacity Constraint
for k in plants.index:
    model += pulp.lpSum([rail_supply[j, k] for j in hubs.index]) <= plants.loc[k, 'plt_capacity'] * plt_status[k]

In [19]:
# Model Status
model.solve()
pulp.LpStatus[model.status]

'Optimal'

## Result

In [20]:
objective = pulp.value(model.objective)

In [21]:
# Hub Supply Table Formulation
rd_output = []
for i, j in road_supply:
    var_output = {
        'county': i,
        'hubs': j,
        'road_supply': road_supply[(i, j)].varValue,
        'hub_status': hub_status[j].varValue
    }
    rd_output.append(var_output)
rd_output_df = pd.DataFrame.from_records(rd_output).sort_values(['county', 'hubs'])
rd_output_df.set_index(['county', 'hubs'], inplace=True)

## Hub Supply Output

In [22]:
rd = []
for i, j in road_supply:
    if rd_output_df.hub_status[i, j] == 1 and rd_output_df.road_supply[i, j] > 0:
        output = {
            'hubs': j,
            'road_supply': rd_output_df.road_supply[i, j],
            'hub_status': rd_output_df.hub_status[i, j] 
        }
        rd.append(output)
rd = pd.DataFrame.from_records(rd).sort_values('hubs')
rd.set_index('hubs', inplace=True)
rd

,road_supply,hub_status
hubs,,
512,300000.000,1.0
17246,300000.000,1.0
17318,300000.000,1.0
17387,300000.000,1.0
17399,300000.000,1.0
17482,300000.000,1.0
17517,300000.000,1.0
17623,300000.000,1.0
17695,63407.767,1.0


In [23]:
# Hub Supply Document Exportation
rd.to_excel('Optimal_Hubs.xlsx')

In [24]:
# Plant Supply Table Formulation
rl_output = []
for j, k in rail_supply: 
    var_output = {
        'hubs': j,
        'plant': k,
        'rail_supply': rail_supply[(j, k)].varValue,
        'plt_status': plt_status[k].varValue
    }
    rl_output.append(var_output)
rl_output_df = pd.DataFrame.from_records(rl_output).sort_values(['hubs', 'plant'])
rl_output_df.set_index(['hubs', 'plant'], inplace=True)

## Plant Supply Output

In [25]:
rl = []
for j, k in rail_supply:
    if rl_output_df.plt_status[j, k] == 1 and rl_output_df.rail_supply[j, k] > 0:
        output = {
            'plants': k,
            'rail_supply': rl_output_df.rail_supply[j, k],
            'plt_status': rl_output_df.plt_status[j, k] 
        }
        rl.append(output)
rl = pd.DataFrame.from_records(rl).sort_values('plants')
rl.set_index('plants', inplace=True)
rl

,rail_supply,plt_status
plants,,
543,655447.00,1.0
9088,655447.00,1.0
9091,655447.00,1.0
9104,655447.00,1.0
9142,655447.00,1.0
9167,464384.73,1.0
9188,655447.00,1.0
9203,655447.00,1.0
10060,655447.00,1.0


In [26]:
# Plant Supply Document Exportation
rl.to_excel('Optimal_Plants.xlsx')